# RDD API 사용하기

In [1]:
from pyspark import SparkConf, SparkContext

In [2]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
myschema = StructType([
         StructField('id', StringType(), True),
         StructField('name', StringType(), True),
         StructField('age', IntegerType(), True),
         StructField('count', IntegerType(), True)
         ])

In [3]:
df = spark.read.format("csv")\
    .schema(myschema)\
  .option("header", "false")\
  .load("./fakefriends.csv")

In [4]:
df.schema

StructType(List(StructField(id,StringType,true),StructField(name,StringType,true),StructField(age,IntegerType,true),StructField(count,IntegerType,true)))

In [5]:
df.show(3)

+---+--------+---+-----+
| id|    name|age|count|
+---+--------+---+-----+
|  0|    Will| 33|  385|
|  1|Jean-Luc| 26|    2|
|  2|    Hugh| 55|  221|
+---+--------+---+-----+
only showing top 3 rows



In [6]:
df.groupby('age').mean().show(50)

+---+--------+------------------+
|age|avg(age)|        avg(count)|
+---+--------+------------------+
| 31|    31.0|            267.25|
| 65|    65.0|             298.2|
| 53|    53.0|222.85714285714286|
| 34|    34.0|             245.5|
| 28|    28.0|             209.1|
| 26|    26.0|242.05882352941177|
| 27|    27.0|           228.125|
| 44|    44.0| 282.1666666666667|
| 22|    22.0|206.42857142857142|
| 47|    47.0|233.22222222222223|
| 52|    52.0| 340.6363636363636|
| 40|    40.0| 250.8235294117647|
| 20|    20.0|             165.0|
| 57|    57.0| 258.8333333333333|
| 54|    54.0| 278.0769230769231|
| 48|    48.0|             281.4|
| 19|    19.0|213.27272727272728|
| 64|    64.0| 281.3333333333333|
| 41|    41.0|268.55555555555554|
| 43|    43.0|230.57142857142858|
| 37|    37.0|249.33333333333334|
| 61|    61.0|256.22222222222223|
| 35|    35.0|           211.625|
| 55|    55.0|295.53846153846155|
| 59|    59.0|             220.0|
| 39|    39.0|169.28571428571428|
| 23|    23.0|

In [7]:
df.groupby('age').sum('count').sort('age').show(1000)

+---+----------+
|age|sum(count)|
+---+----------+
| 18|      2747|
| 19|      2346|
| 20|       825|
| 21|      2807|
| 22|      1445|
| 23|      2463|
| 24|      1169|
| 25|      2172|
| 26|      4115|
| 27|      1825|
| 28|      2091|
| 29|      2591|
| 30|      2594|
| 31|      2138|
| 32|      2287|
| 33|      3904|
| 34|      1473|
| 35|      1693|
| 36|      2466|
| 37|      2244|
| 38|      2903|
| 39|      1185|
| 40|      4264|
| 41|      2417|
| 42|      1821|
| 43|      1614|
| 44|      3386|
| 45|      4024|
| 46|      2908|
| 47|      2099|
| 48|      2814|
| 49|      1108|
| 50|      1273|
| 51|      2115|
| 52|      3747|
| 53|      1560|
| 54|      3615|
| 55|      3842|
| 56|      1840|
| 57|      3106|
| 58|      1282|
| 59|      1980|
| 60|      1419|
| 61|      2306|
| 62|      2870|
| 63|      1536|
| 64|      3376|
| 65|      1491|
| 66|      2488|
| 67|      3434|
| 68|      2696|
| 69|      2352|
+---+----------+



In [13]:
df.groupby('age').mean().sort('age').show(1000)

+---+--------+------------------+
|age|avg(age)|        avg(count)|
+---+--------+------------------+
| 18|    18.0|           343.375|
| 19|    19.0|213.27272727272728|
| 20|    20.0|             165.0|
| 21|    21.0|           350.875|
| 22|    22.0|206.42857142857142|
| 23|    23.0|             246.3|
| 24|    24.0|             233.8|
| 25|    25.0|197.45454545454547|
| 26|    26.0|242.05882352941177|
| 27|    27.0|           228.125|
| 28|    28.0|             209.1|
| 29|    29.0|215.91666666666666|
| 30|    30.0| 235.8181818181818|
| 31|    31.0|            267.25|
| 32|    32.0| 207.9090909090909|
| 33|    33.0| 325.3333333333333|
| 34|    34.0|             245.5|
| 35|    35.0|           211.625|
| 36|    36.0|             246.6|
| 37|    37.0|249.33333333333334|
| 38|    38.0|193.53333333333333|
| 39|    39.0|169.28571428571428|
| 40|    40.0| 250.8235294117647|
| 41|    41.0|268.55555555555554|
| 42|    42.0|             303.5|
| 43|    43.0|230.57142857142858|
| 44|    44.0|

In [8]:
sc = SparkContext.getOrCreate();

In [9]:
#/user/hadoop/fakefriends.csv
lines = sc.textFile("./fakefriends.csv")

In [10]:
lines

./fakefriends.csv MapPartitionsRDD[20] at textFile at NativeMethodAccessorImpl.java:0

In [11]:
def parseLine(line):
    fields = line.split(',')
    age = int(fields[2])
    numFriends = int(fields[3])
    return (age, numFriends)

### map
* map 메서드는 주어진 입력을 원하는 값으로 반환하는 함수를 명시하고 레코드별로 적용

### mapValues
* 생성된 key-value 셋을 사용해 데이터에서 키를 제외하고 값(value)만 추출

In [12]:
rdd = lines.map(parseLine)
totalsByAge = rdd.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
averagesByAge = totalsByAge.mapValues(lambda x: x[0] / x[1])
results = averagesByAge.collect()
for result in results:
    print(result)

(60, 202.71428571428572)
(32, 207.9090909090909)
(68, 269.6)
(38, 193.53333333333333)
(40, 250.8235294117647)
(66, 276.44444444444446)
(26, 242.05882352941177)
(44, 282.1666666666667)
(64, 281.3333333333333)
(54, 278.0769230769231)
(46, 223.69230769230768)
(30, 235.8181818181818)
(56, 306.6666666666667)
(62, 220.76923076923077)
(28, 209.1)
(36, 246.6)
(58, 116.54545454545455)
(20, 165.0)
(18, 343.375)
(52, 340.6363636363636)
(48, 281.4)
(22, 206.42857142857142)
(50, 254.6)
(24, 233.8)
(42, 303.5)
(34, 245.5)
(33, 325.3333333333333)
(55, 295.53846153846155)
(59, 220.0)
(37, 249.33333333333334)
(27, 228.125)
(53, 222.85714285714286)
(57, 258.8333333333333)
(43, 230.57142857142858)
(35, 211.625)
(45, 309.53846153846155)
(67, 214.625)
(19, 213.27272727272728)
(51, 302.14285714285717)
(25, 197.45454545454547)
(21, 350.875)
(49, 184.66666666666666)
(39, 169.28571428571428)
(31, 267.25)
(41, 268.55555555555554)
(69, 235.2)
(65, 298.2)
(61, 256.22222222222223)
(29, 215.91666666666666)
(47, 233